In [1]:
from PIL import Image
from pdb import set_trace
from itertools import repeat

import skimage.morphology
import concurrent.futures
import matplotlib.pyplot as plt
import skimage.measure
import numpy as np
import pandas as pd
import cv2
import glob
import os
import sys
import imageio
import shutil

## Extract dataset metadata

In [2]:
dataset_tif_dir = '/fast/vijay/lung_adenocarcinoma/SCLC2_TIF'
dataset_name = os.path.basename(dataset_tif_dir).split('_')[0]
output_basedir = '/fast/vijay/lung_adenocarcinoma/{}_processed/'.format(dataset_name)
tif_files = glob.glob(os.path.join(dataset_tif_dir, '*.tif'))
os.makedirs(output_basedir, exist_ok=True)
channel_map = {
                'LUAD': {
                    'CD8': 0, 'FoxP3': 1, 'CTLA4': 2, 'CD56': 3, 
                    'Perforin': 4, 'INSM1_CK7': 5, 'DAPI_Core': 6
                },
                'SCLC1': {
                    'CD8': 0, 'FoxP3': 1, 'CTLA4': 2, 'CD56': 3, 
                    'Perforin': 4, 'INSM1_CK7': 5, 'DAPI_Core': 6
                },
                'SCLC2': {
                    'CD8': 0, 'FOXP3': 1, 'CTLA4': 2, 'Perforin': 3, 
                    'CD56': 4, 'INSM1_CK7': 5, 'DAPI_Core': 6
                }
            }

## Extract channels of interest

In [3]:
def extract_channel(img, channel_n, out_dir):
    img.seek(channel_n)
    out_name = os.path.join(out_dir, '{}.png'.format(
                os.path.splitext(os.path.basename(img.filename))[0]))
    Image.fromarray(np.array(img)).convert('RGB').save(out_name)

# Identify TMA regions

## Extract autofluorescent channel

In [4]:
channel_num = channel_map[dataset_name]['DAPI_Core'] + 1
channel_out_dir = os.path.join(output_basedir, 'extracted_channel_{}'.format(channel_num))
os.makedirs(channel_out_dir, exist_ok=True) 
for idx, file_n in enumerate(tif_files): 
    img = Image.open(file_n)
    sys.stdout.write('Extracting channel {} from image {}/{}\r'.format(
        channel_num, idx+1, len(tif_files)))
    sys.stdout.flush()
    try:         
        # extract the last channel (autofluorescence)  
        extract_channel(img, channel_num, channel_out_dir)
    except Exception as e: 
        print(e)

## Mask out TMA region & isolate mispredictions

In [5]:
tma_outdir = os.path.join(output_basedir, 'masks_tma')
os.makedirs(tma_outdir, exist_ok=True)
manual_valdir = os.path.join(output_basedir, 'manual_validationg')
os.makedirs(manual_valdir, exist_ok=True)

In [6]:
def extract_tma_region(file_n):
    img = ~cv2.imread(file_n)
    out_path = '{}.png'.format(os.path.splitext(os.path.basename(file_n))[0])
   
    # threshold image 
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = ~cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    # find contours
    contours = cv2.findContours(cv2.dilate(thresh, 
                    cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15,15))), 
                    cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    cnt_areas = [cv2.contourArea(c) for c in contours]
    sorted_cnt_areas = np.sort(cnt_areas)
    cnt = contours[cnt_areas.index(sorted_cnt_areas[-1])]

    # find tma region
    (x,y),radius = cv2.minEnclosingCircle(cnt)
    center = (int(x), int(y))
    radius = int(radius)
    
    # Mask for noise
    filtered_mask = thresh.copy()
    cv2.circle(filtered_mask, center, radius + 30, (0, 0, 0), -1)
    
    # Check if any TMA region is being excluded
    if np.count_nonzero(filtered_mask) > 3000: 
        return None
    
    # Mask for TMA
    tma_mask = np.zeros(thresh.shape, dtype=np.uint8)
    cv2.circle(tma_mask, center, radius + 30, (255, 255, 255), -1)
    cv2.imwrite(os.path.join(tma_outdir, out_path), tma_mask)
   
    return tma_mask


In [7]:
channel_files = glob.glob('{}/*.png'.format(channel_out_dir))

# # Extract masks in parallel
# executor = concurrent.futures.ProcessPoolExecutor(max_workers=8)
# tma_masks = executor.map(extract_tma_region, files)
# for i in tma_masks: print(i)

for file_n in channel_files:
    tma_mask = extract_tma_region(file_n)
    if tma_mask is None: shutil.copy(file_n, os.path.join(output_basedir, 'manual_validationg'))

# Remove autofluorescence from CD8 channel

## Extract CD8 channel

In [8]:
channel_num = channel_map[dataset_name]['CD8']
cd8_out_dir = os.path.join(output_basedir, 'extracted_channel_{}'.format(channel_num))
os.makedirs(cd8_out_dir, exist_ok=True) 
for idx, file_n in enumerate(tif_files): 
    img = Image.open(file_n)
    sys.stdout.write('Extracting channel {} from image {}/{}\r'.format(
        channel_num, idx+1, len(tif_files)))
    sys.stdout.flush()
    try: 
        # extract the CD8 channel
        extract_channel(img, channel_num, cd8_out_dir)
    except Exception as e: 
        print(e)

## Extract DAPI channel

In [9]:
channel_num = channel_map[dataset_name]['DAPI_Core']
dapi_out_dir = os.path.join(output_basedir, 'extracted_channel_{}'.format(channel_num))
os.makedirs(dapi_out_dir, exist_ok=True) 
for idx, file_n in enumerate(tif_files): 
    img = Image.open(file_n)
    sys.stdout.write('Extracting channel {} from image {}/{}\r'.format(
        channel_num, idx+1, len(tif_files)))
    sys.stdout.flush()
    try: 
        # extract the DAPI channel  
        extract_channel(img, channel_num, dapi_out_dir)
    except Exception as e: 
        print(e)

## Dilate DAPI channel & mask CD8+ cells

In [10]:
''' Dilate DAPI and mask the CD8 cells'''
cd8_mask_outdir = os.path.join(output_basedir, 'cd8_autofluorescence_masksg')
os.makedirs(cd8_mask_outdir, exist_ok=True)
for cd8_file, dapi_file in zip(sorted(glob.glob('{}/*.png'.format(cd8_out_dir))),
                               sorted(glob.glob('{}/*.png'.format(dapi_out_dir)))):
    if (os.path.basename(cd8_file).split('-')[0] != os.path.basename(cd8_file).split('-')[0]): continue
    
    # CD8 channel
    img1 = ~cv2.imread(cd8_file)
    gray_img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    thresh1 = ~cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # DAPI channel
    img = ~cv2.imread(dapi_file)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = ~cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    thresh = cv2.dilate(thresh, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))

    # CD8 mask
    cd8_mask = cv2.bitwise_and(img1, img1, mask=~cv2.bitwise_and(~thresh, thresh1))
    out_path = '{}.png'.format(os.path.splitext(os.path.basename(cd8_file))[0])
    
    # Save mask
    cv2.imwrite(os.path.join(cd8_mask_outdir, out_path), ~cv2.bitwise_and(~thresh, thresh1))

## Plot masks in a grid

In [11]:
hstacked_img = []
vstacked_img = []

for idx, i in enumerate(glob.glob('{}/*.png'.format(manual_valdir))):
    img = cv2.resize(cv2.imread(i), dsize=None, fx=0.25, fy=0.25)
    vstacked_img.append(np.vstack((img, np.ones((5, img.shape[1], 3))* 255)))
    if ((idx + 1) % 10 == 0): 
        vstacked_img = np.vstack(vstacked_img)
        hstacked_img.append(vstacked_img)
        hstacked_img.append(np.ones((vstacked_img.shape[0], 5, 3))* 255)
        vstacked_img = []
        
if len(vstacked_img) and len(vstacked_img) != 10: 
    for i in range(10 - len(vstacked_img)): vstacked_img.append(np.zeros_like(vstacked_img[0]))
        
if len(vstacked_img): 
    vstacked_img = np.vstack(vstacked_img)
    hstacked_img.append(vstacked_img)
    hstacked_img.append(np.ones((vstacked_img.shape[0], 5, 3))* 255)

cv2.imwrite(os.path.join(output_basedir, 'manual_validationg.png'), np.hstack(hstacked_img))

True

In [12]:
out_zip_name = '{}/{}_masks'.format(output_basedir, dataset_name)
shutil.make_archive(out_zip_name, 'zip', 
                    os.path.join(output_basedir, 'masks_tma'), 
                    os.path.join(output_basedir, 'cd8_autofluorescence_masksg'))

'/fast/vijay/lung_adenocarcinoma/SCLC2_processed/SCLC2_masks.zip'